## DECODING GOOGLE NEWS ARTICLES 


In [1]:
import pandas as pd
#import geopandas as gpd
import json
from ast import literal_eval
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import json
import base64
import requests
import bs4
from bs4 import BeautifulSoup
import re
#import lxml.etree

#pip install geopandas
#pip install spacy
#python -m spacy download en_core_web_sm
#pip install lxml
#pip install parser-libraries 


In [2]:
"""Decode encoded Google News entry URLs."""
import base64
import functools
import re

# Ref: https://stackoverflow.com/a/59023463/

_ENCODED_URL_PREFIX = "https://news.google.com/articles/"
_ENCODED_URL_RE = re.compile(fr"^{re.escape(_ENCODED_URL_PREFIX)}(?P<encoded_url>[^?]+)")
_DECODED_URL_RE = re.compile(rb'^\x08\x13".+?(?P<primary_url>http[^\xd2]+)\xd2\x01')


@functools.lru_cache(2048)
def _decode_google_news_url(url: str) -> str:
    match = _ENCODED_URL_RE.match(url)
    encoded_text = match.groupdict()["encoded_url"]  # type: ignore
    encoded_text += "==="  # Fix incorrect padding. Ref: https://stackoverflow.com/a/49459036/
    decoded_text = base64.urlsafe_b64decode(encoded_text)

    match = _DECODED_URL_RE.match(decoded_text)
    primary_url = match.groupdict()["primary_url"]  # type: ignore
    primary_url = primary_url.decode()
    return primary_url


def decode_google_news_url(url: str) -> str:  # Not cached because not all Google News URLs are encoded.
    """Return Google News entry URLs after decoding their encoding as applicable."""
    return _decode_google_news_url(url) if url.startswith(_ENCODED_URL_PREFIX) else url

In [40]:
rz = pd.read_csv("full_rizz.csv", sep = ";")

In [41]:
# Create an empty list for the urls
urls = []

# loop through the urls in the column that contains links to googlenews articles
for url in rz['link']:
    try:
        new = decode_google_news_url(url)
        urls.append(new)
    
    # Attribute error caused by a faulty link will be removed from dataset
    except AttributeError:
        rz.drop(rz.loc[rz.link == url].index, inplace = True)

# After dropping rows we will need to reset the index
rz = rz.reset_index(drop=True)

In [43]:
# The lengths of the list and dataframe will be the same, as will their order
# We can concatenate into a new dataframe
rz = pd.concat([rz, pd.DataFrame({'url':urls})], axis=1)
# Store the dataframe 
rz.to_csv("updated_urls_rizz.csv", sep = ';', index = False)

## SHARECOUNT RETRIEVAL HERE

In [2]:
rz = pd.read_csv("csv/updated_urls_rizz.csv", sep = ";")

In [3]:
rz = rz.drop(columns = 'Unnamed: 0')

In [4]:
rz.columns

Index(['common_name', 'scientific_name', 'distribution', 'iucn_status',
       'date_saved', 'Kingdom', 'Class_x', 'Order', 'Family', 'Genus',
       'Species', 'Listing', 'NativeDistributionFullNames',
       'Extinct_Distribution', 'Common name', 'General ID name', 'Count',
       'Class_y', 'Ecosystem', 'Appendix', 'Appendix exceptions, if any',
       'text', 'title', 'link', 'image', 'date', 'ner', 'url'],
      dtype='object')

In [5]:
# From Sharecount API 
# 500 quota per day 
from requests.structures import CaseInsensitiveDict

sharecount = []
for url in rz['url'][1999:]: 
    
    api = "https://api.sharedcount.com/v1.1?apikey=2a0336186dc8f5740fef9c1eb81947d66119c29f&url=" + url
    
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    resp = requests.get(api, headers=headers)
    details = resp.json()
    #for n in range(len(details)):
        #count = details[n]['Facebook']['share_count']
        #print(count)
 
    
    fb_count = details['Facebook']['share_count']
    sharecount.append(fb_count)
        
   # except KeyError:
       # print(url, details)
        #rz.drop(rz.loc[rz.url == url].index, inplace = True)
    


In [6]:
frst = pd.read_csv("csv/urls_df_1999.csv", sep = ";")

In [7]:
frst.head()

,common_name,scientific_name,distribution,iucn_status,date_saved,Kingdom,Class_x,Order,Family,Genus,...,Appendix,"Appendix exceptions, if any",text,title,link,image,date,ner,url,sharecount
0,Javan Rhinoceros,Rhinoceros sondaicus,"Bangladesh, Cambodia, China, India, Indonesia,...",Critically Endangered,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,1,NaN,"TEMPO.CO, Jakarta - Indonesia is home to two o...",Sumatran Rhino Population Critical,https://news.google.com/articles/CBMiQmh0dHBzO...,https://statik.tempo.co/data/2015/02/09/id_369...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Indonesia is ho...",https://en.tempo.co/read/806444/sumatran-rhino...,0.0
1,Javan Rhinoceros,Rhinoceros sondaicus,"Bangladesh, Cambodia, China, India, Indonesia,...",Critically Endangered,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,1,NaN,"TEMPO.CO, Jakarta - Director of the Biodiversi...",Javan Rhinos to be Transferred to Cikepuh Wild...,https://news.google.com/articles/CBMiWmh0dHBzO...,https://statik.tempo.co/data/2011/12/29/id_100...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Director of the...",https://en.tempo.co/read/878302/javan-rhinos-t...,0.0
2,Grevy's Zebra,Equus grevyi,"Djibouti, Ethiopia, Kenya, Somalia, South Sudan",EN,2020-06-18,Animalia,Mammalia,Perissodactyla,Equidae,Equus,...,1,NaN,Share\n\n\n\n\n\n\n\n\n\n\n\nA Grevy’s Zebra f...,California zoo welcomes Grevy’s zebra colt foal,https://news.google.com/articles/CBMiSGh0dHBzO...,https://i1.wp.com/www.horsetalk.co.nz/wp-conte...,2001-11-30,"[{'sent': 'A Grevy’s', 'ents': {'ORG': ['Grevy...",https://www.horsetalk.co.nz/2014/06/05/califor...,0.0
3,Tucuxi,Sotalia fluviatilis,"Brazil, Colombia, Costa Rica, Ecuador, Nicarag...",Data Deficient,2020-06-18,Animalia,Mammalia,Cetacea,Delphinidae,Sotalia,...,1,NaN,At the Institute for Marine Mammal Studies in ...,Why dolphins are deep thinkers,https://news.google.com/articles/CBMiQGh0dHBzO...,https://i.guim.co.uk/img/static/sys-images/Env...,2003-07-03,[{'sent': 'At the Institute for Marine Mammal ...,https://www.theguardian.com/science/2003/jul/0...,7823.0
4,Western Gorilla,Gorilla gorilla,"Angola, Cameroon, Central African Republic, Co...",CR,2020-06-18,Animalia,Mammalia,Primates,Hominidae,Gorilla,...,1,NaN,By Helen Briggs\n\nBBC News Online science rep...,Gorilla study gives social clues,https://news.google.com/articles/CBMiNWh0dHA6L...,http://newsimg.bbc.co.uk/media/images/39953000...,2004-03-23,"[{'sent': 'By Helen Briggs\n\n', 'ents': {'PER...",http://news.bbc.co.uk/2/hi/science/nature/3557...,10.0


In [18]:
len(sharecount)

500

In [12]:
pd.Series(sharecount).set_axis(range(1999, ))

ValueError: Length mismatch: Expected axis has 500 elements, new values have 1999 elements

In [11]:
frst[1999:]

,common_name,scientific_name,distribution,iucn_status,date_saved,Kingdom,Class_x,Order,Family,Genus,...,Appendix,"Appendix exceptions, if any",text,title,link,image,date,ner,url,sharecount
1999,Fish-eating Crocodile,Gavialis gangeticus,"Bangladesh, Bhutan, India, Myanmar, Nepal, Pak...",CR,2020-06-18,Animalia,Reptilia,Crocodylia,Gavialidae,Gavialis,...,1,NaN,New Delhi | Jagran News Desk: National Endange...,National Endangered Species Day 2020: Five rar...,https://news.google.com/articles/CBMiiAFodHRwc...,https://www.jagranimages.com/images/2020/may/e...,2020-05-14,"[{'sent': 'New Delhi | Jagran News Desk:', 'en...",https://english.jagran.com/india/national-enda...,NaN
2000,African Elephant,Loxodonta africana,"Angola, Benin, Botswana, Burkina Faso, Burundi...",Vulnerable,2020-06-18,Animalia,Mammalia,Proboscidea,Elephantidae,Loxodonta,...,1,"Except populations of Botswana, Namibia, South...",There are few photographers whose work is as i...,Joel Sartore on creating the Photo Ark: May 15...,https://news.google.com/articles/CBMihQFodHRwc...,https://static.nationalgeographic.co.uk/files/...,2020-05-14,[{'sent': 'There are few photographers whose w...,https://www.nationalgeographic.co.uk/family/20...,NaN
2001,Gharial,Gavialis gangeticus,"Bangladesh, Bhutan, India, Myanmar, Nepal, Pak...",CR,2020-06-18,Animalia,Reptilia,Crocodylia,Gavialidae,Gavialis,...,1,NaN,Himalayan News Service\n\nShare Now:\n\n\n\n\n...,Nepal establishes border outpost near Kalapani,https://news.google.com/articles/CBMiUmh0dHA6L...,https://thehimalayantimes.com/wp-content/uploa...,2020-05-14,"[{'sent': 'Himalayan News Service\n\n', 'ents'...",http://thehimalayantimes.com/nepal/nepal-estab...,NaN
2002,Irrawaddy Dolphin,Orcaella brevirostris,"Australia, Bangladesh, Brunei Darussalam, Camb...",EN,2020-06-18,Animalia,Mammalia,Cetacea,Delphinidae,Orcaella,...,1,NaN,Efforts urged as nation’s wildlife threatened\...,Efforts urged as nation’s wildlife threatened,https://news.google.com/articles/CBMiUWh0dHBzO...,https://www.khmertimeskh.com/wp-content/upload...,2020-05-14,[{'sent': 'Efforts urged as nation’s wildlife ...,https://www.khmertimeskh.com/723288/efforts-ur...,NaN
2003,White Rhinoceros,Ceratotherium simum,"Angola, Botswana, Central African Republic, Ch...",Near Threatened,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Ceratotherium,...,1,NaN,Calistoga artist crusades for endangered speci...,Calistoga artist crusades for endangered species,https://news.google.com/articles/CBMiYWh0dHBzO...,https://www.pressdemocrat.com/csp/mediapool/si...,2020-05-14,[{'sent': 'Calistoga artist crusades for endan...,https://www.pressdemocrat.com/entertainment/10...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,Grizzly Bear,Ursus arctos,"Afghanistan, Albania, Armenia, Austria, Azerba...",Least Concern,2020-06-18,Animalia,Mammalia,Carnivora,Ursidae,Ursus,...,1,"Only the populations of: Bhutan, China, Mexico...",Volunteer to help with crowds at Wyoming grizz...,Volunteer to help with crowds at Wyoming grizz...,https://news.google.com/articles/CBMiaWh0dHBzO...,https://s.hdnux.com/photos/01/12/22/44/1947472...,2020-05-30,[{'sent': 'Volunteer to help with crowds at Wy...,https://www.michigansthumb.com/news/article/Vo...,NaN
2189,Greater one-horned rhino,Rhinoceros unicornis,"Bangladesh, Bhutan, India, Myanmar, Nepal, Pak...",VU,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,1,NaN,We are going to consider a few outstanding thr...,Three-Dot Lounge Coronavirus Edition,https://news.google.com/articles/CBMiU2h0dHBzO...,https://i0.wp.com/www.escondidograpevine.com/w...,2020-05-30,[{'sent': 'We are going to consider a few outs...,https://www.escondidograpevine.com/2020/05/30/...,NaN
2190,Fin Whale,Balaenoptera physalus,"Algeria, Angola, Antarctica, Argentina, Austra...",VU,2020-06-18,Animalia,Mammalia,Cetacea,Balaenopteridae,Balaenoptera,...,1,NaN,A 12-metre-long fin whale which washed up at H...,Fin wh

In [9]:
frst['sharecount'].iloc[1499:1999].fillna(pd.Series(sharecount).set_axis(range(1499, 1999)), inplace = True)

In [10]:
frst.to_csv('csv/urls_df_1999.csv', sep = ';', index = False)

In [62]:
## Directly retrieveing from Facebook
# Has lower request limit

from requests.structures import CaseInsensitiveDict
import requests

sharecount = []
for url in urls:
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    token = "EAAIxJUBe0zYBAOKUVYhZCHWRlZBm0Pj4M8DGka4LvaFuyeKCMLF8NG2rGtamLjwAMZBSrMFWPY6ZAnpZAihDhP7wjs9drlJ4W8C1ZCCyguI2Kg8rxgfUVrv22sFrcNLXYS1PYZAZALqr2ZB4bLpKMYHC7yGA7lXF8tf3G6BOI6YdQArlLt5bzY6TMZC73HTBr4ztpbRkSefUXGFZB9TTgbyZBL8nY8sSq5YuZAFy9owthwenPOMvK3K9ZACtP5hBiY2GfV4GcZD"
    resp = requests.get(f'https://graph.facebook.com/?id={url}&fields=engagement&access_token={token}', headers=headers).json()
    #print(resp)
    
    #break
    try:
        fb_shares = (resp['engagement']['share_count'])
        sharecount.append(fb_shares)
    except KeyError:
        print("API Limit exceeded")
        
        break
    


API Limit exceeded


In [14]:
pd.concat([frst, pd.DataFrame({'sharecount':sharecount})], axis=1)

,common_name,scientific_name,distribution,iucn_status,date_saved,Kingdom,Class_x,Order,Family,Genus,...,"Appendix exceptions, if any",text,title,link,image,date,ner,url,sharecount,sharecount
0,Javan Rhinoceros,Rhinoceros sondaicus,"Bangladesh, Cambodia, China, India, Indonesia,...",Critically Endangered,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,NaN,"TEMPO.CO, Jakarta - Indonesia is home to two o...",Sumatran Rhino Population Critical,https://news.google.com/articles/CBMiQmh0dHBzO...,https://statik.tempo.co/data/2015/02/09/id_369...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Indonesia is ho...",https://en.tempo.co/read/806444/sumatran-rhino...,0.0,187.0
1,Javan Rhinoceros,Rhinoceros sondaicus,"Bangladesh, Cambodia, China, India, Indonesia,...",Critically Endangered,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,NaN,"TEMPO.CO, Jakarta - Director of the Biodiversi...",Javan Rhinos to be Transferred to Cikepuh Wild...,https://news.google.com/articles/CBMiWmh0dHBzO...,https://statik.tempo.co/data/2011/12/29/id_100...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Director of the...",https://en.tempo.co/read/878302/javan-rhinos-t...,0.0,1.0
2,Grevy's Zebra,Equus grevyi,"Djibouti, Ethiopia, Kenya, Somalia, South Sudan",EN,2020-06-18,Animalia,Mammalia,Perissodactyla,Equidae,Equus,...,NaN,Share\n\n\n\n\n\n\n\n\n\n\n\nA Grevy’s Zebra f...,California zoo welcomes Grevy’s zebra colt foal,https://news.google.com/articles/CBMiSGh0dHBzO...,https://i1.wp.com/www.horsetalk.co.nz/wp-conte...,2001-11-30,"[{'sent': 'A Grevy’s', 'ents': {'ORG': ['Grevy...",https://www.horsetalk.co.nz/2014/06/05/califor...,0.0,114.0
3,Tucuxi,Sotalia fluviatilis,"Brazil, Colombia, Costa Rica, Ecuador, Nicarag...",Data Deficient,2020-06-18,Animalia,Mammalia,Cetacea,Delphinidae,Sotalia,...,NaN,At the Institute for Marine Mammal Studies in ...,Why dolphins are deep thinkers,https://news.google.com/articles/CBMiQGh0dHBzO...,https://i.guim.co.uk/img/static/sys-images/Env...,2003-07-03,[{'sent': 'At the Institute for Marine Mammal ...,https://www.theguardian.com/science/2003/jul/0...,7823.0,282.0
4,Western Gorilla,Gorilla gorilla,"Angola, Cameroon, Central African Republic, Co...",CR,2020-06-18,Animalia,Mammalia,Primates,Hominidae,Gorilla,...,NaN,By Helen Briggs\n\nBBC News Online science rep...,Gorilla study gives social clues,https://news.google.com/articles/CBMiNWh0dHA6L...,http://newsimg.bbc.co.uk/media/images/39953000...,2004-03-23,"[{'sent': 'By Helen Briggs\n\n', 'ents': {'PER...",http://news.bbc.co.uk/2/hi/science/nature/3557...,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,Grizzly Bear,Ursus arctos,"Afghanistan, Albania, Armenia, Austria, Azerba...",Least Concern,2020-06-18,Animalia,Mammalia,Carnivora,Ursidae,Ursus,...,"Only the populations of: Bhutan, China, Mexico...",Volunteer to help with crowds at Wyoming grizz...,Volunteer to help with crowds at Wyoming grizz...,https://news.google.com/articles/CBMiaWh0dHBzO...,https://s.hdnux.com/photos/01/12/22/44/1947472...,2020-05-30,[{'sent': 'Volunteer to help with crowds at Wy...,https://www.michigansthumb.com/news/article/Vo...,NaN,NaN
2189,Greater one-horned rhino,Rhinoceros unicornis,"Bangladesh, Bhutan, India, Myanmar, Nepal, Pak...",VU,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,NaN,We are going to consider a few outstanding thr...,Three-Dot Lounge Coronavirus Edition,https://news.google.com/articles/CBMiU2h0dHBzO...,https://i0.wp.com/www.escondidograpevine.com/w...,2020-05-30,[{'sent': 'We are going to consider a few outs...,https://www.escondidograpevine.com/2020/05/30/...,NaN,NaN
2190,Fin Whale,Balaenoptera physalus,"Algeria, Angola, Antarctica, Argentina, Austra...",VU,2020-06-18,Animalia,Mammalia,Cetacea,Balaenopteridae,Balaenoptera,...,NaN,A 12-metre-long fin whale which washed up at H...,Fin whale washed up on Essex beach has been re...,https://news.google.com/articl